In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime

import cifar10

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [69]:
# Constants describing the training process.
img_size_cropped =24
num_channels = 3
num_classes = 10

batch_size = 256
num_iterations = 2
print_unit = 1
learning_rate = 1e-4

In [67]:
# Download and Import Data

data_path = "data/CIFAR-10/"
data_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
cifar10.maybe_download_and_extract(url=data_url, download_dir=data_path)

images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

## For Debugging
# images_train = images_train[0:100,:,:,:]
# images_test = images_test[0:10,:,:,:]
# labels_train = labels_train[0:100,:]
# labels_test = labels_test[0:10,:]

Data has apparently already been downloaded and unpacked.
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [ ]:
# Data Augmentation
def pre_process_image(image, training):

    if training:
        
        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])
        
        image = tf.image.random_flip_left_right(image)

        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)

    else:

        image = tf.image.resize_image_with_crop_or_pad(image,
                                                       target_height=img_size_cropped,
                                                       target_width=img_size_cropped)

    return image

def pre_process(images, training):

    images = tf.map_fn(lambda image: pre_process_image(image, training), images)

    return images

In [ ]:
# Shuffle
def random_batch():
    # Number of images in the training-set.
    num_images = len(images_train)
    
    # Create a random index.
    idx = np.random.choice(num_images,
                           size=batch_size,
                           replace=False)

    # Use the random index to select random images and labels.
    x_batch = images_train[idx,:,:,:]
    y_batch = labels_train[idx, :]

    return x_batch, y_batch

In [70]:
# Inference

def _variable_with_weight_decay(name, shape, stddev, wd):
    """
    Helper to create an initialized Variable with weight decay

    Args:
        name: name of the variable
        shape: list of ints
        stddev: standard deviation of a truncated Gaussian
        wd: add L2Loss weight decay multiplied by this float. If None, weight
            decay is not added for this Variable.

    Returns:
        Variable Tensor
    """
    var = _variable_on_cpu(
        name,
        shape,
        tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var


def _variable_on_cpu(name, shape, initializer):
    """
    Helper to create a Variable stored on CPU memory

    Args:
        name: name of the variable
        shape: list of ints
        initializer: initializer for Variable

    Returns:
        Variable Tensor

    """
    with tf.device('/cpu:0'):
        var = tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)
    return var

def compute_logits_cnn(x):
    
    phase_train = tf.placeholder(tf.bool, name='phase_train')

    x = tf.reshape(x, [-1, img_size_cropped, img_size_cropped, 3])

    # layer_conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[5, 5, 3, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)

    # max_pool
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')

    # layer_conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[5, 5, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name) 

    # max_pool
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
    
    # layer_fc1
    with tf.variable_scope('fc1') as scope:
        # flatten
        reshape = tf.reshape(pool2, [-1, 6*6*64])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 256],
                                              stddev=0.04, wd=0.0)
        biases = _variable_on_cpu('biases', [256], tf.constant_initializer(0.0))
        fc1 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)

    # layer_fc2
    with tf.variable_scope('fc2') as scope:
        weights = _variable_with_weight_decay('weights', shape=[256, 128],
                                              stddev=0.04, wd=0.0)
        biases = _variable_on_cpu('biases', [128], tf.constant_initializer(0.0))
        fc2 = tf.nn.relu(tf.matmul(fc1, weights) + biases, name=scope.name)

    # softmax_classifier
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [128, num_classes],
                                          stddev=1/128.0, wd=0.0)
        biases = _variable_on_cpu('biases', [num_classes],
                              tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(fc2, weights), biases, name=scope.name)   

    return softmax_linear

In [21]:
# loss
def compute_cross_entropy(logits, y):
    sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits, name='cross_ent_terms')
    cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

In [ ]:
# accuracy
def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

In [75]:
# TRAIN
now = datetime.now()
dir_name = 'log_{0}_{1}'.format(now.month,now.day)
export_dir = 'savedmodel_{0}_{1}_{2}_{3}_{4}'.format(now.month,now.day,now.hour,now.minute,now.second)
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Graph().as_default():
    global_step = tf.train.get_or_create_global_step()

    with tf.device('/cpu:0'):
        x = tf.placeholder(tf.float32, shape=[None, 32, 32, num_channels], name='x')
        y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')

        images = pre_process(images=x, training=True)

    with tf.name_scope('model'):
        logits = compute_logits_cnn(images)

    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y_true)

    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits=logits, y=y_true)

    with tf.name_scope('opt'):
        opt = tf.train.AdamOptimizer(learning_rate)
        train_step = opt.minimize(loss)

    with tf.name_scope('summaries'):
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.histogram('logit', logits)
        tf.summary.image('input', tf.reshape(images, [-1, img_size_cropped, img_size_cropped, num_channels]))
        summary_op = tf.summary.merge_all()

    saver = tf.train.Saver()

    sess = tf.Session()
    summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
    summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
    summary_writer_val = tf.summary.FileWriter(dir_name+'/val')

    try:
        print("Trying to restore last checkpoint ...")

        # Use TensorFlow to find the latest checkpoint - if any.
        last_chk_path = tf.train.latest_checkpoint(checkpoint_dir='./model/')

        # Try and load the data in the checkpoint.
        saver.restore(sess, save_path=last_chk_path)

        # If we get to this point, the checkpoint was successfully loaded.
        print("Restored checkpoint from:", last_chk_path)
    except:
        # If the above failed for some reason, simply
        # initialize all the variables for the TensorFlow graph.
        print("Failed to restore checkpoint. Initializing variables instead.")
        sess.run(tf.global_variables_initializer())
        
    for i in range(num_iterations):
        
        X_batch, y_batch = random_batch()

        _, summary = sess.run((train_step, summary_op),
                                feed_dict={x: X_batch, y_true: y_batch})

        if i%print_unit==0:
            summary_writer_train.add_summary(summary, i)
                
            (train_error, train_accuracy, train_logits) = sess.run((loss, accuracy, logits), {x: X_batch, y_true: y_batch})
            print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_accuracy), flush=True)
            print("\rStep {0:3d}: training loss {1:0.4f}".format(i, train_error), flush=True)
           
            (val_error, val_accuracy, summary) = sess.run((loss, accuracy,summary_op), {x:images_test, y_true:labels_test})
            print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_accuracy), flush=True)
            print("\rStep {0:3d}: val loss {1:0.4f}".format(i, val_error), flush=True)
            summary_writer_val.add_summary(summary, i)
            saver.save(sess, './model/model_iter', global_step=i)

    save_path = saver.save(sess, "./model/final_model.ckpt")
    builder.add_meta_graph_and_variables(sess, ["foo-tag"])

builder.save()

Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from ./model/final_model.ckpt


INFO:tensorflow:Restoring parameters from ./model/final_model.ckpt


Restored checkpoint from: ./model/final_model.ckpt
Step   0: training accuracy 0.0859
Step   0: training loss 2.3036
Step   0: val accuracy 0.1000
Step   0: val loss 2.3026
Step   1: training accuracy 0.1445
Step   1: training loss 2.3013
Step   1: val accuracy 0.1000
Step   1: val loss 2.3026
INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: b'savedmodel_12_14_14_29_3/saved_model.pb'


INFO:tensorflow:SavedModel written to: b'savedmodel_12_14_14_29_3/saved_model.pb'


b'savedmodel_12_14_14_29_3/saved_model.pb'